# Setup

In [ ]:
!apt-get update
!apt-get install git

#!pip install scikit-learn===0.24.0 #needed for autopytorch but need >=1.0 needed for handcrafted feature generation or get https://github.com/mne-tools/mne-features/issues/73
!pip install scikit-learn --upgrade

# MNE-BIDS preprocessing functions
!pip install mne
!pip install MNE-bids
#!pip install -r https://raw.githubusercontent.com/mne-tools/mne-bids-pipeline/main/requirements.txt
!pip install typing_extensions psutil joblib dask[distributed] jupyter-server-proxy scikit-learn pandas seaborn json_tricks coloredlogs python-picard fire pyqt5 pyvista pyvistaqt openpyxl

!git clone https://github.com/mne-tools/mne-bids-pipeline.git /cloned-mne-bids-pipeline

!pip install AutoReject
!pip install coffeine
!pip install Braindecode

#!rm -r /cloned-paper-repo
!git clone https://github.com/meeg-ml-benchmarks/brain-age-benchmark-paper.git /cloned-paper-repo

!pip install -U matplotlib

!pip install -U scipy
!pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html

!pip install numpy matplotlib scipy numba scikit-learn mne PyWavelets pandas

#!apt-get install swig -y
#!pip install Cython numpy
#!pip install auto-sklearn



### Restart Runtime 

make sure all updated packages are loaded in their newest versions


In [ ]:
import os
def restart_runtime():
  os.kill(os.getpid(), 9)
restart_runtime()

###Mount Google Drive for access to data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
######################################################################################################
#  ________   _________   ___    ___ ___  ___  ________  ________  ___  __      _______   _______
# |\   ___  \|\___   ___\|\  \  /  /|\  \|\  \|\   __  \|\   ____\|\  \|\  \   /  ___  \ /  ___  \
# \ \  \\ \  \|___ \  \_|\ \  \/  / | \  \\\  \ \  \|\  \ \  \___|\ \  \/  /|_/__/|_/  //__/|_/  /|
#  \ \  \\ \  \   \ \  \  \ \    / / \ \   __  \ \   __  \ \  \    \ \   ___  \__|//  / /__|//  / /
#   \ \  \\ \  \   \ \  \  /     \/   \ \  \ \  \ \  \ \  \ \  \____\ \  \\ \  \  /  /_/__  /  /_/__
#    \ \__\\ \__\   \ \__\/  /\   \    \ \__\ \__\ \__\ \__\ \_______\ \__\\ \__\|\________\\________\
#     \|__| \|__|    \|__/__/ /\ __\    \|__|\|__|\|__|\|__|\|_______|\|__| \|__| \|_______|\|_______|
#                        |__|/ \|__|
######################################################################################################

import mne
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import make_pipeline
from mne.decoding import Vectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge

# Path to  data
test_path = "/content/drive/My Drive/brainAge_competition/finalStage_testSet_raws/"
condition = "EC"  # use only closed eyes condition for demonstration purpose

finalStage_test_raws = []
for s in range(1601, 2001):
  fname = f"subj{s:04}_{condition}_raw.fif.gz"
  raw = mne.io.read_raw(test_path + fname, preload=True)
  finalStage_test_raws.append(raw)


# Preprocessing like how we did w/ original age prediction paper

In [ ]:
for r in finalStage_test_raws:
  r.resample(sfreq=250) 
  r.filter(l_freq = 0.1, h_freq = 49)

import pickle
%cd /content/drive/My Drive/brainAge_competition/
filename = 'finalStage_test_preproc.sav' 
pickle.dump(finalStage_test_raws, open(filename, 'wb'))


/content/drive/My Drive/brainAge_competition


## pick chans, epoch

In [ ]:
%cd /content/drive/My Drive/brainAge_competition/
with open(r"finalStage_test_preproc.sav", "rb") as input_file:
  finalStage_test_preproc = pickle.load(input_file)

#################### pick channels ############################

chans_22subset = ['E36', 'E104', 'E24', 'E124', 'E33', 'E122', 'E22', 'E9', 
         'E14', 'E21', 'E15', 'E11', 'E70', 'E83', 'E52', 'E92', 'E58', 
         'E96', 'E45', 'E108', 'E72', 'E62']

for r in finalStage_test_preproc:
  r.pick_channels(chans_22subset)

finalStage_test_preproc[399]

####################### 2 second epochs #############################

events = mne.make_fixed_length_events(
            finalStage_test_preproc[0], 
            #id=3000, 
            start=2,
            #duration=rest_epochs_duration,
            duration = 2 - 1 / finalStage_test_preproc[0].info['sfreq'],
            #overlap=rest_epochs_overlap,
            overlap = 0.,
            stop=38,
            )

finalStage_testepoched_data = [None] * len(finalStage_test_preproc)
len(finalStage_testepoched_data)
for i in range(len(finalStage_test_preproc)):
  print(i)
  inputdat =  finalStage_test_preproc[i]
  finalStage_testepoched_data[i] = mne.Epochs(inputdat, events=events, event_id=1,
                      tmin=0, tmax=2,
                      proj=False, 
                      baseline=None,
                      preload=True, 
                      #decim=decim,
                      #metadata=metadata,
                      #event_repeated=event_repeated,
                      reject=None
                      )


(18, 22, 501)

# Generate Handcrafted Features

In [ ]:
hc_selected_funcs = [
    'std',
    'kurtosis',
    'skewness',
    'quantile',
    'ptp_amp',
    'mean',
    'pow_freq_bands',
    'spect_entropy',
    'app_entropy',
    'samp_entropy',
    'svd_entropy',
    'hurst_exp',
    'hjorth_complexity',
    'hjorth_mobility',
    'line_length',
    'wavelet_coef_energy',
    'higuchi_fd',
    'zero_crossings',
    'svd_fisher_info'
]
hc_func_params = {
    'quantile__q': [0.1, 0.25, 0.75, 0.9],
    'pow_freq_bands__freq_bands': [0, 2, 4, 8, 13, 18, 24, 30, 49],
    'pow_freq_bands__ratios': 'all',
    'pow_freq_bands__ratios_triu': True,
    'pow_freq_bands__log': True,
    'pow_freq_bands__normalize': None,
}

!pip install numpy matplotlib scipy numba scikit-learn mne PyWavelets pandas
!pip install mne-features
!pip install h5io
from mne_features.feature_extraction import extract_features

finalStage_testout = [None] * len(finalStage_testepoched_data)

for i in range(len(finalStage_testepoched_data)):
  print(i)
  input = finalStage_testepoched_data[i]
  inputDat = input.get_data()
  inputSfreq = input.info['sfreq']
  inputChans = input.ch_names

  finalStage_testout[i] = extract_features(
      inputDat, 
      inputSfreq, 
      hc_selected_funcs,
      funcs_params=hc_func_params, 
      n_jobs=-1, 
      ch_names=inputChans,
  )


finalStage_testEpochs_averaged = np.vstack([x.mean(axis=0) for x in finalStage_testout])

############# Any nan ? #############

print(np.any(np.isnan(finalStage_testEpochs_averaged))) #nope

############### any infs #################

print(np.any(np.isinf(finalStage_testEpochs_averaged))) #nope

%cd /content/drive/My Drive/brainAge_competition/
filename = 'finalStage_testData_250hz_2secEpochs_handcraftedFeatures.sav' 
pickle.dump(finalStage_testEpochs_averaged, open(filename, 'wb'))


/content/drive/My Drive/brainAge_competition


# Generate Filterbank Riemann Features

In [ ]:
import coffeine

frequency_bands = {
    "low": (0.1, 1),
    "delta": (1, 4),
    "theta": (4.0, 8.0),
    "alpha": (8.0, 15.0),
    "beta_low": (15.0, 26.0),
    "beta_mid": (26.0, 35.0),
    "beta_high": (35.0, 49)
}

def extract_fb_covs(epochs, n_jobs):
    features, meta_info = coffeine.compute_features(
        epochs, features=('covs',), n_fft=1024, n_overlap=512,
        fs=epochs.info['sfreq'], fmax=49, frequency_bands=frequency_bands, n_jobs=n_jobs)
    features['meta_info'] = meta_info
    return features

finalStage_testout = [None] * len(finalStage_testepoched_data)
len(finalStage_testout)
for i in range(0,400,1):
  print(i)
  input = finalStage_testepoched_data[i]

  finalStage_testout[i] = extract_fb_covs(
      epochs=input,
      n_jobs=4
  )


%cd /content/drive/My Drive/brainAge_competition/
filename = 'finalStage_testData_250hz_2secEpochs_filterbankRiemannFeatures.sav' 
pickle.dump(finalStage_testout, open(filename, 'wb'))


/content/drive/My Drive/brainAge_competition


# Age predictions based on fb riemann features

In [ ]:
# load train set ###################################################

%cd /content/drive/My Drive/brainAge_competition/
with open(r"trainData_250hz_2secEpochs_filterbankRiemannFeatures.sav", "rb") as input_file:
  trainout = pickle.load(input_file)

covs = [sub['covs'] for sub in trainout]
covs = np.array(covs)
X = pd.DataFrame(
    {band: list(covs[:, ii]) for ii, band in
      enumerate(frequency_bands)})

X_trainSet = X[0:900]
X_validSet = X[900:]

# load test set ########################################################
finalStage_fbriemannFeats_test = finalStage_testout

finalStage_testcovs = [sub['covs'] for sub in finalStage_fbriemannFeats_test]
len(finalStage_testcovs)
finalStage_testcovs[0].shape
finalStage_testcovs = np.array(finalStage_testcovs)
finalStage_testX = pd.DataFrame(
    {band: list(finalStage_testcovs[:, ii]) for ii, band in
      enumerate(frequency_bands)})

# load train ys ##################################################
train_path = "/content/drive/My Drive/brainAge_competition/raws/"
train_subj = 1200
meta = pd.read_csv(train_path + "train_subjects.csv")
y_train = []
for age in meta["age"][:train_subj]:
    y_train.append(age)
y_train

y_trainSet = y_train[0:900]
y_validSet = y_train[900:]

# age predictions ##################################################
analyze_channels = 22
rank = analyze_channels - 1
import coffeine
filter_bank_transformer = coffeine.make_filter_bank_transformer(
    names=list(frequency_bands),
    method='riemann',
    projection_params=dict(scale='auto', n_compo=rank)
)

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
model = make_pipeline(
    filter_bank_transformer, StandardScaler(),
    RidgeCV(alphas=np.logspace(-5, 10, 100)))

model.fit(X_trainSet,y_trainSet)

train_predictions = model.predict(X_trainSet)
result = sklearn.metrics.r2_score(y_trainSet,train_predictions)
print(f"train R2: {result}")
mae_result = sklearn.metrics.mean_absolute_error(y_trainSet,train_predictions)
print(f"train MAE = {mae_result}")

valid_predictions = model.predict(X_validSet)
result = sklearn.metrics.r2_score(y_validSet,valid_predictions)
print(f"valid R2: {result}")
mae_result = sklearn.metrics.mean_absolute_error(y_validSet,valid_predictions)
print(f"valid MAE = {mae_result}")

finalStage_test_predictions = model.predict(finalStage_testX)
finalStage_test_predictions

import pickle
%cd /content/drive/My Drive/brainAge_competition/
filename = 'finalStage_testData_filterbankRiemann_ridgeCV_predictions.sav' 
pickle.dump(finalStage_test_predictions, open(filename, 'wb'))


/content/drive/My Drive/brainAge_competition
